In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
!pip -q install faiss-cpu

In [42]:
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
from datasets import Image as IMG
from typing import List, Union, Tuple
from transformers import CLIPProcessor, CLIPModel

In [43]:
import faiss

In [44]:
import glob

img_path = '/kaggle/input/genhincharacters'
img_files = glob.glob(img_path + "/*.[jJpP][pPnN][gG]")

In [45]:
img_files = sorted(img_files)
img_files[0]

'/kaggle/input/genhincharacters/Albedo.png'

In [46]:
device = "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [47]:
def encode_images(
    images: Union[List[str], List[PIL.Image.Image]],
    batch_size: int
):
    def transform_func(x):
        if isinstance(x['image'], PIL.Image.Image):
            image = x['image']
        else:
            image = [IMG().decode_example(img) for img in x['image']]
        return processor(images=image, return_tensors='pt')

    dataset = Dataset.from_dict({'image' : images})
    dataset = dataset.cast_column('image', IMG(decode=False)) \
                if isinstance(images[0], str) \
                else dataset
    dataset.set_format('torch')
    dataset.set_transform(transform_func)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    image_embeddings = []
    progress = tqdm(total=len(images)//batch_size, position=0)
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k : v.to(device) for k,v in batch.items()}
            image_embeddings.extend(
                model.get_image_features(**batch).detach().cpu().numpy()
            )
            progress.update(1)
        progress.close()
    return np.stack(image_embeddings)

In [48]:
vector_embedding = np.array(encode_images(img_files, 7))

14it [00:09,  1.51it/s]                        
